GPU Setting -> GPU 1번 사용

In [ ]:
import tensorflow as tf
cpus = tf.config.experimental.list_physical_devices('CPU')
if cpus:
    try:
        tf.config.experimental.set_visible_devices(cpus[0], 'CPU')
    except RuntimeError as e:
        print(e)

In [9]:
import tensorflow as tf
visible_devices = tf.config.get_visible_devices()
print(visible_devices)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


Transformer 모델의 구성 -> Transformer_Implement.ipynb

In [13]:
import import_ipynb
import Transformer_Implement as ti
from CommonModule.Handle_Dir import mkdir_p, del_folder

import re
import os
import numpy as np
from pathlib import Path
from gensim.models import Word2Vec, KeyedVectors

importing Jupyter notebook from /data/ksb/TestDir/articleSummary-Jupyter/Transformer/CommonModule/Handle_Dir.ipynb


경로 설정

In [11]:
BASE_DIR = "/data/ksb/TestDir"
DATA_BASE_DIR = os.path.join(BASE_DIR, 'sample_articles')

PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Preprocessed-Data")
MINI_PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Mini-Preprocessed-Data")

SUMMARY_PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Summary-Preprocessed-Data")
MINI_SUMMARY_PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Mini-Summary-Preprocessed-Data")

MODEL_BASE_DIR = os.path.join(Path(os.getcwd()).parent, 'Word-Embedding-Model')
word2vec_model_path = os.path.join(MODEL_BASE_DIR, 'word2vec-256.model')
wv_model_path = os.path.join(MODEL_BASE_DIR, 'word2vec-256.wordvectors')

말뭉치와 함께 임베딩 모델 불러오기

In [5]:
word2vec = Word2Vec.load(word2vec_model_path)
wv = KeyedVectors.load(wv_model_path, mmap='r')

In [7]:
def get_embedded_list(filepath, wv):

    embedding_vec_list = []
    for idx, proc_path in enumerate(iglob(os.path.join(filepath, '**.csv'), recursive=False)):
    
        f = open(proc_path, 'r', newline="\n", encoding="utf-8")
            
        for [article_idx, title, contents] in csv.reader(f):  
            content = contents.split("\t")    
            vec = np.array([wv.word_vec(token) for sent in content for token in sent.split() if token in wv])
            embedding_vec_list.append(vec)

    return embedding_vec_list

In [8]:
D_MODEL = 256
LAYER_NUM = 6
NUM_HEADS = 8
DFF = 512
VOCAB_SIZE = wv.vectors.shape[0]

BATCH_SIZE = 64
BUFFER_SIZE = 20000

WARMUP_STEPS = 50
EPOCHS = 70

전처리 된 59,604개의 기사 본문 데이터를 임베딩 벡터로 변환한다

각 기사마다 토큰 개수에 따라 (None, 256)의 Matrix를 가진다.

각 기사의 토큰에 대해 임베딩을 수행한 결과 리스트를 `train_embedded_list`, `target_embedded_list`로 저장한다.

In [ ]:
train_embedded_list = get_embedded_list(MINI_PREPROCESSED_PATH, wv)
target_embedded_list = get_embedded_list(MINI_SUMMARY_PREPROCESSED_PATH, wv)

In [ ]:
get_max_length = lambda x : np.max([len(x[idx]) for idx in range(len(x))])
MAX_LEN = get_max_length(train_embedded_list)
MAX_LEN

In [ ]:
target_max_len = get_max_length(target_embedded_list)
MAX_LEN = target_max_len if target_max_len > MAX_LEN else MAX_LEN
MAX_LEN

각 문장의 토큰을 임베딩 벡터로 변환한다.  

각 인코딩된 값을 임베딩 벡터 값으로 대체한 후에는 (59604, `max_token`, 256)의 크기를 가지는 Tensor가 된다.  
- `max_token`은 각 기사 본문의 토큰 수 중 가장 큰 값을 의미한다.

이의 결과로 (59604, 4392, 256)의 dimension을 가진다.

In [ ]:
train_embedded_matrix = tf.keras.preprocessing.sequence.pad_sequences(train_embedded_list, maxlen=40, padding='post', dtype=np.float32)
target_embedded_matrix = tf.keras.preprocessing.sequence.pad_sequences(target_embedded_list, maxlen=40, padding='post', dtype=np.float32)

print("Train Embedded Matrix shape : {train} \nTargetEmbedded Matrix shape : {target}"\
      .format(train=train_embedded_matrix.shape,
             target=target_embedded_matrix.shape))

`dataset` 구조  
```
({'encoder_inputs' : [[[...]]], 'decoder_inputs' : [[[...]]] }, { 'Output' : [[[...]]] }) 의 튜플이 59604개 있다.
```


In [ ]:
decoder_inputs = target_embedded_matrix
decoder_inputs[:, -1, :] = 0.0

outputs = target_embedded_matrix
outputs[:, 1, :] = 0.0


dataset = tf.data.Dataset.from_tensor_slices((
    {
        'encoder_inputs': train_embedded_matrix, # Encoder Input
        'decoder_inputs': decoder_inputs # Decoder Input
    },
    {
        # Decoder Output, Remove <SOS>
        'Output': outputs 
    },
))

In [ ]:
for dict_1, dict_2 in dataset.as_numpy_iterator():
    print("encoder input : {enc}, decoder input : {dec}".format(enc= dict_1['encoder_inputs'].shape,
                                                               dec=dict_1['decoder_inputs'].shape))
    print("output shape : {}".format(dict_2['Output'].shape))

데이터의 59604를 `BATCH_SIZE(=64)`에 따라 분할하며, 
dataset의 크기는 아래와 같다.

In [ ]:
# 데이터셋을 메모리 또는 로컬 저장소에 캐시, 각 epoch 동안 실행되는 일부 작업(파일 열기 및 데이터 읽기 등)이 저장됨
dataset = dataset.cache() 
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
for dict_1, dict_2 in dataset.as_numpy_iterator():    
    print("encoder input : {enc}, decoder input : {dec}".format(enc= dict_1['encoder_inputs'].shape,
                                                               dec=dict_1['decoder_inputs'].shape))
    print("output shape : {}".format(dict_2['Output'].shape))

학습률 Learning Rate 조정

In [ ]:
lrate_scheduler = ti.LearningRate(d_model=D_MODEL, warmup_steps=WARMUP_STEPS)

Optimizer 정의

In [ ]:
beta_1 = 0.9  
beta_2 = 0.98
epsilon = 10 ** -9

optimizer = tf.keras.optimizers.Adam(lrate_scheduler, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

Transformer 모델 정의

In [ ]:
with tf.device('/CPU:0'):
    model = ti.Transformer(
        vocab_size=VOCAB_SIZE,
        layer_num=LAYER_NUM,
        dff=DFF,
        d_model=D_MODEL,
        num_heads=NUM_HEADS).get_transformer()

In [ ]:
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=10, min_lr=0.00005, verbose=1)

model.fit(dataset, batch_size=16, epochs=EPOCHS, shuffle=True, callbacks=[reduce_lr])